In [1]:
# If you have not done so already: pip install sklearn-crfsuite
import sklearn_crfsuite
import pandas as pd
import glob
from csv import DictReader



In [2]:
filename = './../../Data/parc_features/parc_dev_features.tsv'



In [5]:
with open(filename, 'r') as infile:
    dict_reader = DictReader(infile, delimiter='\t')
    list_of_dict = list(dict_reader)

for row in list_of_dict:
    print(row['dependency_label'])


det
nmod:poss
case
nsubj:pass
aux
aux:pass
root
case
amod
compound
obl
det
obl:tmod
punct
case
obl
case
nmod
punct
conj
punct
conj
cc
conj
punct
det
advmod
amod
nsubj
aux
cop
det
compound
compound
compound
root
case
fixed
obl:tmod
punct
det
compound
nsubj:pass
aux:pass
root
mark
xcomp
case
advmod
obl
compound
nummod
case
nmod:poss
case
obl
compound
nummod
punct
case
fixed
det
obl
case
compound
nmod
punct
det
appos
case
compound
nmod
punct
compound
appos
punct
nmod:poss
case
nsubj:pass
case
det
compound
compound
compound
nmod
aux:pass
root
mark
xcomp
punct
mark
advmod
advmod
advcl
case
det
nummod
compound
obl
acl
obl:tmod
case
det
compound
compound
obl
punct
det
nsubj:pass
aux
aux:pass
root
case
det
obl
det
compound
nsubj
cop
advmod
acl:relcl
case
nmod:poss
case
obl
punct
mark
nsubj:pass
aux:pass
advcl
case
nmod:poss
amod
compound
obl
punct
nsubj:pass
aux:pass
root
case
fixed
advmod
amod
compound
obl
nsubj
ccomp
mark
xcomp
case
compound
obl
case
amod
nmod
cc
compound
conj
punct
advmod
c

advmod
nsubj
acl:relcl
det
amod
amod
obj
nsubj
acl:relcl
obj
mark
advcl
det
obj
case
nmod:poss
compound
nmod
cc
advmod
advmod
conj
nmod:poss
obj
cc
conj
obj
case
obl
punct
mark
advcl
cc
conj
punct
compound
obj
punct
punct
nsubj
aux
root
advmod
advmod
obj
mark
nsubj
flat
advcl
case
det
amod
obl
punct
det
compound
nsubj
cop
det
advmod
amod
root
compound
nummod
case
nmod
compound
compound
nmod
punct
cc
aux
aux
cop
advmod
conj
aux
advmod
det
nsubj:pass
case
det
compound
compound
nmod
aux:pass
conj
case
det
obl
case
punct
nmod
punct
punct
det
amod
amod
appos
punct
det
amod
nummod
nsubj
root
case
det
obl
case
amod
nmod
case
nummod
amod
amod
nmod
cc
case
amod
conj
case
det
compound
nmod
punct
det
nsubj
root
det
amod
obj
case
punct
compound
nmod:poss
case
punct
nmod
mark
advcl
obj
punct
cc
det
amod
conj
case
nmod
punct
case
nmod
punct
nmod:poss
case
compound
compound
nsubj
punct
appos
flat
punct
aux
root
mark
xcomp
advmod
obj
punct
nsubj
cop
advmod
root
punct
advcl
nummod
compound
compound
obj

punct
det
amod
amod
appos
case
compound
compound
nmod
punct
compound
nsubj
cop
advmod
root
punct
det
nsubj:pass
aux:pass
parataxis
cc
nsubj
conj
nmod:poss
obj
advmod
case
obl
punct
det
compound
nsubj:pass
case
det
compound
nummod
compound
nmod
aux
aux
aux:pass
ccomp
case
obl
case
det
obl
case
nmod
case
compound
nmod
obj
nsubj
acl:relcl
case
obl
case
compound
nmod
cc
amod
conj
punct
det
compound
nsubj
root
punct
amod
obl:tmod
punct
nsubj
root
det
amod
obj
case
nmod
nsubj
acl:relcl
amod
obj
mark
det:predet
det
nummod
nsubj:pass
aux
aux:pass
advcl
punct
advmod
case
obl
punct
amod
compound
nsubj
root
advmod
amod
obj
punct
compound
compound
nsubj
root
det
nummod
compound
obj
case
amod
compound
nmod
cc
det
nummod
compound
conj
case
amod
compound
nmod
punct
compound
nsubj
root
det
nummod
compound
obj
case
compound
nmod
punct
advcl
nmod:poss
advmod
amod
obj
det
obl:tmod
punct
advmod
punct
nsubj
case
nmod
punct
case
det
nmod:poss
case
amod
nmod
punct
root
nummod
obj
punct
compound
compound
nmod

case
case
det
nmod
punct
expl
root
advmod
det
nsubj
case
amod
nmod
punct
advcl
mark
xcomp
compound:prt
case
det
amod
obl
case
nmod
punct
det
amod
compound
nsubj
acl
punct
det
xcomp
case
nmod
punct
root
mark
xcomp
amod
obj
case
amod
nmod
cc
conj
case
case
det
nmod
punct
expl
root
det
nsubj
case
nmod
cc
det
compound
conj
punct
cc
mark
advcl
amod
obj
punct
compound
amod
nsubj
root
det
obj
case
amod
compound
obl
punct
det
compound
compound
nsubj
cop
root
det
obl:tmod
case
obl
punct
punct
nsubj
aux
ccomp
case
det:predet
det
obl
punct
punct
root
nsubj
flat
punct
det
appos
punct
punct
det
root
nsubj
acl:relcl
mark
xcomp
obj
case
det
nmod
punct
nsubj
case
amod
punct
amod
nmod
punct
punct
case
nmod
punct
root
compound:prt
case
obl
case
nmod
punct
conj
det
obj
case
det
nmod
amod
punct
conj
nmod:poss
obj
punct
cc
conj
mark
xcomp
punct
advmod
mark
advcl
cc
conj
amod
obj
advmod
punct
det
compound
nsubj
root
mark
advcl
punct
cc
fixed
punct
advmod
case
det
obl
case
det
nmod
case
amod
nmod
punct
advmo

nsubj
flat
advmod
acl:relcl
punct
compound
compound
nsubj
root
nsubj:pass
flat
punct
nummod
appos
amod
punct
nsubj
aux
cop
acl:relcl
cc
amod
compound
conj
case
det
amod
obl
punct
aux:pass
ccomp
amod
amod
xcomp
case
det
compound
nmod
punct
nsubj
root
nmod:poss
obj
punct
appos
flat
flat
punct
nsubj
acl:relcl
det
obj
cc
conj
xcomp
punct
nsubj:pass
flat
flat
flat
punct
nmod:npmod
punct
nsubj
acl:relcl
det
nmod:poss
case
compound
obj
punct
aux:pass
root
obj
cc
amod
compound
conj
punct
advcl
obj
flat
punct
compound
compound
compound
compound
nsubj
root
nsubj
ccomp
advmod
nummod
obj
case
compound
compound
compound
nmod
case
advmod
obl
compound
nummod
case
compound
compound
compound
nmod
punct
amod
nsubj
root
nsubj
advmod
ccomp
nummod
compound
amod
obj
case
nmod:poss
case
amod
nmod
punct
nsubj
root
det
obj
case
amod
nmod
case
obl
cc
conj
punct
nsubj
punct
case
case
obl
punct
appos
punct
root
compound
obj
case
amod
compound
nmod
cc
amod
compound
conj
punct
case
det
obl
acl
obl:tmod
nummod
punct

...

aux:pass
conj
advmod
mark
det
nsubj
advcl
punct
advmod
fixed
nummod
nsubj:pass
case
nmod
det
nmod:tmod
aux
aux:pass
root
case
det
obl
case
det
amod
nummod
obl
punct
det
amod
amod
nsubj
case
nmod
flat
punct
appos
punct
nsubj
acl:relcl
advmod
fixed
nummod
obj
case
nmod
advmod
punct
aux
root
mark
xcomp
punct
nsubj
root
det
nsubj
aux
advmod
ccomp
compound
obj
case
amod
compound
obl
case
nmod:poss
compound
compound
compound
compound
nmod
punct
cc
aux
conj
det
obj
case
nmod
acl
case
det
compound
obl
punct
amod
compound
nsubj
punct
case
nmod
case
det
amod
nmod
punct
root
amod
obj
punct
compound
compound
nsubj
aux
root
det
nmod:poss
case
obj
advcl
advmod
case
obl
punct
det
amod
nsubj
case
compound
compound
nmod
punct
acl
case
advmod
nummod
obl
case
det
compound
nummod
punct
amod
compound
punct
nmod
case
det
compound
compound
nmod
punct
root
det
compound
obj
acl
case
obl
case
det
nmod
punct
nsubj
nsubj
acl:relcl
compound:prt
det
obj
case
nmod
cc
conj
det
obj
aux
root
det
obj
punct
cc
advmod
con

aux
root
mark
nsubj:pass
aux:pass
ccomp
advmod
amod
compound
obl:tmod
punct
case
det
compound
obl
punct
cc
conj
det
compound
compound
nummod
amod
punct
compound
punct
obj
case
nmod
punct
conj
flat
compound
conj
cc
det
compound
compound
nummod
conj
case
compound
compound
compound
nmod
punct
advmod
punct
mark
advcl
det
amod
compound
compound
obj
cc
conj
amod
obj
punct
nsubj
root
nsubj
cop
punct
advmod
ccomp
punct
mark
nsubj
aux
cop
ccomp
mark
xcomp
obj
compound
nummod
case
compound
nmod
case
nmod
punct
case
det
amod
obl
case
det
compound
compound
nmod
punct
nmod:poss
case
nsubj
cop
advmod
advmod
det
amod
root
punct
mark
det
nsubj
aux
advcl
mark
aux:pass
xcomp
case
nmod:poss
case
compound
obl
aux
aux:pass
conj
advmod
punct
compound
nsubj
punct
advmod
punct
root
nsubj
ccomp
mark
xcomp
obj
compound
nummod
case
compound
obl
case
nmod
punct
case
det
compound
obl
punct
det
nsubj
case
compound
compound
nmod
root
obl:tmod
mark
nsubj:pass
aux
aux:pass
ccomp
case
nummod
amod
obl
obj
nsubj
acl:relc

punct
amod
nsubj
root
punct
compound
nsubj
punct
case
obl
punct
root
det
amod
nummod
obj
case
obl
cc
det
amod
conj
case
nmod
punct
advmod
punct
amod
nsubj
cc
amod
conj
aux
advmod
advcl
punct
cop
nmod:poss
case
root
punct
root
advmod
advmod
case
amod
obl
cc
conj
punct
appos
case
det
advmod
amod
punct
conj
cc
advmod
amod
amod
nmod
case
det
nmod
punct
advmod
fixed
punct
det
nsubj
aux
root
obj
cc
conj
case
det
amod
nummod
obl
case
det
amod
obl
punct
cc
advmod
root
mark
cop
det
xcomp
mark
acl
punct
compound
nmod:poss
case
nsubj
aux
root
cc
advmod
conj
case
det
obl
case
nmod:poss
nummod
obl
punct
case
det
obl
punct
nsubj
cop
root
advmod
advmod
case
nummod
obl
case
det
nummod
nmod
cc
compound
conj
cop
advmod
nummod
conj
punct
det
amod
nsubj
root
advmod
advmod
amod
obj
punct
compound
nsubj
root
nummod
obj
punct
det
compound
nsubj
root
advmod
fixed
nummod
obj
case
advmod
det
compound
obl
punct
cc
advmod
fixed
nummod
nsubj
case
compound
nmod
cop
conj
punct
case
det
obl
punct
det
amod
compound
ns

nsubj
cc
nmod:poss
conj
acl:relcl
case
det
amod
obl
case
nmod:poss
flat
case
obl
advmod
case
compound
nmod
punct
det
appos
punct
case
obl
case
nmod
punct
nsubj
flat
root
nsubj
aux
ccomp
mark
xcomp
case
det
obl
case
det
obl
cc
mark
det
nsubj
punct
conj
case
det
obl
punct
punct
nsubj
flat
punct
nmod:poss
compound
appos
cc
conj
advmod
root
obj
nummod
case
nmod:poss
flat
case
amod
obl
case
det
obl
punct
case
fixed
compound
obl
punct
cc
nsubj
flat
root
nsubj
flat
ccomp
compound:prt
nmod:poss
obj
advmod
case
obl
punct
mark
det
nsubj
advcl
mark
xcomp
det
compound
nmod:poss
case
obj
advmod
advmod
mark
nsubj
flat
advcl
punct
punct
nsubj
ccomp
advmod
xcomp
case
obl
punct
conj
nmod:poss
obj
cc
conj
det
obj
case
nmod
mark
nsubj
cop
det
ccomp
punct
punct
nsubj
flat
root
punct
case
obl
punct
compound
nsubj
root
mark
nsubj
flat
ccomp
obj
nummod
case
amod
nmod
case
nummod
amod
compound
compound
obl
case
obl
cc
conj
case
det
obl
case
nmod
flat
punct
nsubj
acl:relcl
mark
csubj
det
obj
cop
punct
det
advm

compound
compound
compound
obl
case
det
compound
case
compound
nmod
nmod
punct
det
nsubj
cc
nmod:poss
conj
root
det
obj
punct
case
det
amod
obl
punct
compound
nsubj
root
det
compound
nsubj
aux
ccomp
det
obj
mark
advcl
case
amod
obl
case
det
amod
nummod
obl
punct
det
compound
nsubj
advmod
root
det
obj
obl:tmod
nummod
punct
nummod
punct
cc
aux
aux
conj
det
obj
advmod
advmod
punct
det
nsubj
root
nsubj
aux
ccomp
mark
xcomp
det
obj
case
det
nmod
punct
compound
nsubj
root
det
nummod
compound
obj
case
amod
amod
nmod
punct
advmod
case
fixed
det
amod
obl
case
amod
nmod
punct
det
amod
nsubj
cc
compound
conj
punct
root
nsubj
ccomp
obj
case
nmod
compound
compound
punct
cc
nummod
conj
punct
advmod
nummod
nmod:npmod
case
obl
compound
nummod
punct
cc
nummod
conj
det
appos
punct
det
amod
nsubj
root
det
obj
case
nmod
compound
nummod
punct
amod
case
nummod
obl
det
obl
case
det
amod
nmod
punct
case
det
obl
mark
acl
det
compound
amod
obj
case
compound
nmod
case
compound
nmod
punct
aux
advmod
root
case
det